In [75]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
import uuid
import shutil
import glob
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img , img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from sklearn.metrics import classification_report, accuracy_score

In [76]:
tf.keras.backend.clear_session()

In [77]:
# # Define data augmentation parameters (adjust as needed)
# datagen = ImageDataGenerator(
#     rotation_range=10,  # Rotate images by up to 40 degrees
#     shear_range=0.15,     # Shear by up to 20%
#     zoom_range=0.2,      # Zoom by up to 20%
#     horizontal_flip=True, # Randomly flip images horizontally
#     brightness_range=(0.5, 1.5),  # Adjust brightness range
#     channel_shift_range=60  # Shift color channels by up to 50 units
# )

# # Define path to your image folder
# image_dir = "Train/Suit"

# # Define output directory for augmented images (create if it doesn't exist)
# save_to_dir = "Train/Suit"
# import os
# os.makedirs(save_to_dir, exist_ok=True)  # Create directory if it doesn't exist

# # Loop through each image in the folder
# for filename in os.listdir(image_dir):
#     # Load the image
#     img = load_img(os.path.join(image_dir, filename))

#     # Convert to array
#     x = img_to_array(img)

#     # Reshape (optional, based on your model's requirements)
#     x = x.reshape((1,) + x.shape)  # Reshape if your model expects a specific format

#     # Generate and save augmented images
#     i = 0
#     for batch in datagen.flow(x, batch_size=1, save_to_dir=save_to_dir, save_prefix="aug_", save_format="jpeg"):
#         i += 1
#         # You can adjust the number of augmented images per original image here (e.g., i > 5)
#         if i > 2:
#             break
# print(f"Successfully augmented and saved images to {save_to_dir}")


In [78]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)

In [79]:
x = tf.keras.layers.GlobalAvgPool2D()(base_model.output)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64,activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(9,activation="softmax")(x)

model = tf.keras.Model(inputs = base_model.input , outputs = x)


In [80]:
#freezing the original model's layers
for layer in base_model.layers:
    layer.trainable = False

In [81]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss=CategoricalCrossentropy(from_logits=True),  # Adjust if needed
              metrics=['accuracy'])

In [82]:
train_Path = "Train"
IMAGE_SIZE = (224,224)
folders = glob.glob(train_Path + '/*')
folders

['Train\\Dress',
 'Train\\Jacket',
 'Train\\Pants',
 'Train\\Shoes',
 'Train\\Short',
 'Train\\Suit',
 'Train\\Sunglasses',
 'Train\\T_shirt',
 'Train\\Watch']

In [83]:
labels = ["Dress","Jacket","Pants","Shoes","Short","Suit","Sunglasses","T_shirt","Watch"]

In [84]:
train_datagen = ImageDataGenerator(
    rescale = 1./225,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)
training_set = train_datagen.flow_from_directory(
    train_Path,
    target_size = IMAGE_SIZE,
    batch_size = 32,
    class_mode = 'categorical', # As we have more than 2 so using categorical.. for 2 we might have used binary.
    shuffle = True
)


Found 11637 images belonging to 9 classes.


In [106]:
test_datagen = ImageDataGenerator(rescale=1./225)
test_Path = "Test"
test_set = test_datagen.flow_from_directory(
    test_Path,  # Use your test data directory here
    target_size=IMAGE_SIZE,
    batch_size=32,
    class_mode='categorical',
    shuffle = False
)

Found 720 images belonging to 9 classes.


In [86]:
model.fit(training_set, epochs=10, steps_per_epoch=len(training_set),validation_data=test_set)

Epoch 1/10


c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\keras\src\backend\tensorflow\nn.py:559: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


364/364 ━━━━━━━━━━━━━━━━━━━━ 250s 664ms/step - accuracy: 0.7837 - loss: 0.6524 - val_accuracy: 0.9583 - val_loss: 0.1343
Epoch 2/10


c:\Users\ghass\anaconda3\envs\yolo\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


364/364 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9583 - val_loss: 0.1374
Epoch 3/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 246s 668ms/step - accuracy: 0.9560 - loss: 0.1324 - val_accuracy: 0.9653 - val_loss: 0.1478
Epoch 4/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9653 - val_loss: 0.1328
Epoch 5/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 238s 648ms/step - accuracy: 0.9659 - loss: 0.1039 - val_accuracy: 0.9597 - val_loss: 0.1399
Epoch 6/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9597 - val_loss: 0.1319
Epoch 7/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 267s 724ms/step - accuracy: 0.9752 - loss: 0.0769 - val_accuracy: 0.9569 - val_loss: 0.1603
Epoch 8/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.9569 - val_loss: 0.1534
Epoch 9/10
364/364 ━━━━━━━━━━━━━━━━━━━━ 270s 732ms/step - accuracy

In [101]:
# Define path to your image
image_path = "t-shirt.jpg"

# Load the image
img = load_img(image_path, target_size=IMAGE_SIZE)  # Adjust if needed

# Convert to array and normalize
x = img_to_array(img)
x = np.expand_dims(x, axis=0)  # Add a batch dimension as the model expects batches
x = x / 255.0  # Normalize pixel values (assuming rescale used during training)


In [102]:
# Make predictions
predictions = model.predict(x)

# Get the class with the highest probability
predicted_class = np.argmax(predictions[0])  # Assuming one prediction per image
max = np.max(predictions[0])
if(max<0.7):
    print("Other")
else:
    print(labels[predicted_class])
print(predictions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
T_shirt
[[1.4495128e-02 1.3699923e-05 1.3049570e-04 1.8745908e-05 1.0433465e-04
  4.1262661e-05 7.9836865e-04 9.8425215e-01 1.4579509e-04]]


In [104]:
# model.save("ClothesModelV1.keras")

In [107]:
test_labels = test_set.classes

# Predict the labels for the test set
predictions = model.predict(test_set, steps=len(test_set))
predicted_classes = np.argmax(predictions, axis=1)

# Get the class indices to labels mapping
class_labels = list(test_set.class_indices.keys())

# Print classification report
report = classification_report(test_labels, predicted_classes, target_names=class_labels)
print(report)

c:\Users\ghass\anaconda3\envs\yolo\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


23/23 ━━━━━━━━━━━━━━━━━━━━ 9s 386ms/step
              precision    recall  f1-score   support

       Dress       0.90      1.00      0.95        54
      Jacket       0.99      0.90      0.94       129
       Pants       0.93      0.94      0.94        69
       Shoes       0.98      0.98      0.98       120
       Short       0.97      0.92      0.94        72
        Suit       0.82      1.00      0.90        23
  Sunglasses       0.96      0.99      0.97        73
     T_shirt       0.93      0.94      0.93        96
       Watch       0.99      1.00      0.99        84

    accuracy                           0.96       720
   macro avg       0.94      0.96      0.95       720
weighted avg       0.96      0.96      0.96       720

